In [2]:
import numpy
import numpy as np
import pandas as pd
import os

FOLDER_NAME = "HnM"
FILE_NAME_TRANSACTIONS = "transactions_train.csv"
FILE_NAME_ARTICLES = "articles.csv"
SAVE_FOLDER = 'Narrowed_HnM'
SAVE_LOCATION = f'{FOLDER_NAME}/{SAVE_FOLDER}/{FILE_NAME_TRANSACTIONS.split(".")[0]}'
if not os.path.isdir(f"{FOLDER_NAME}/{SAVE_FOLDER}"):
    os.mkdir(f'{FOLDER_NAME}/{SAVE_FOLDER}')
TOP_N = 0 # Positive Integer for top n items, 0 for hand-picked values
Y_VALUE_PERIOD_DAYS = 30

In [3]:
transactions_df = pd.read_csv(f"{FOLDER_NAME}/{FILE_NAME_TRANSACTIONS}", dtype={"article_id": "str"})
articles_df = pd.read_csv(f"{FOLDER_NAME}/{FILE_NAME_ARTICLES}", dtype={"article_id": "str"})

In [4]:
transactions_df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,0.006763,1


In [ ]:
import time
import datetime

def create_apk_evaluation_subset(transactions, days):
    transactions_x_values = transactions_df.copy()
    transactions_x_values["timestamp"] = transactions["t_dat"].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()))
    now_timestamp = transactions_x_values["timestamp"].max()
    transactions_y_values = transactions_x_values[transactions_x_values["timestamp"] >= (now_timestamp - 60*60*24*days)]
    transactions_x_values = transactions_x_values[transactions_x_values["timestamp"] < (now_timestamp - 60*60*24*days)]
    print(transactions.shape, transactions_x_values.shape, transactions_y_values.shape)

    transactions_x_values["timestamp"] = transactions_x_values["timestamp"].apply(lambda x: (now_timestamp - x) // (60*60*24))
    transactions_x_articles = transactions_x_values.groupby("customer_id")["article_id"].apply(list).reset_index()
    transactions_x_timestamps = transactions_x_values.groupby("customer_id")["timestamp"].apply(list).reset_index()
    transactions_x_values = pd.merge(transactions_x_articles, transactions_x_timestamps, how="inner", on="customer_id")

    transactions_y_values["timestamp"] = transactions_y_values["timestamp"].apply(lambda x: (now_timestamp - x) // (60*60*24))
    transactions_y_articles = transactions_y_values.groupby("customer_id")["article_id"].apply(list).reset_index()
    transactions_y_timestamps = transactions_y_values.groupby("customer_id")["timestamp"].apply(list).reset_index()
    transactions_y_values = pd.merge(transactions_y_articles, transactions_y_timestamps, how="inner", on="customer_id")

    transactions_processed = pd.merge(transactions_x_values, transactions_y_values, how="outer", on="customer_id")
    print(transactions_processed.shape, transactions_x_values.shape, transactions_y_values.shape)
    # transactions["timestamp"] = transactions["t_dat"].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()))
    # now_timestamp = transactions["timestamp"].max()
    # transactions["days_since_transaction"] = transactions["timestamp"].apply(lambda x: (now_timestamp - x) // (60*60*24))
    # transactions_subset = transactions[transactions["timestamp"] > (transactions["timestamp"].max() - 60*60*24*days)]
    # #transactions_y_values = transactions_subset.groupby("customer_id")["article_id"].aggregate(lambda x: " ".join(x)).reset_index()
    # transactions_y_values = transactions_subset.groupby("customer_id")["article_id"].apply(list).aggregate(lambda x: x).reset_index()
    # transactions_y_timestamp = transactions_subset.groupby("customer_id")["days_since_transaction"].apply(list).aggregate(lambda x: x).reset_index()
    # transactions_y_values = pd.merge(transactions_y_values, transactions_y_timestamp, how="inner", on="customer_id")
    #
    # transactions_subset_train = transactions[transactions["timestamp"] < (transactions["timestamp"].max() - 60*60*24*days)]
    # transactions_x_values = transactions_subset_train.groupby("customer_id")["article_id"].apply(list).aggregate(lambda x: x).reset_index()
    # transactions_x_timestamps = transactions_subset_train.groupby("customer_id")["days_since_transaction"].apply(list).aggregate(lambda x: x).reset_index()
    # transactions_x_values = pd.merge(transactions_x_values, transactions_x_timestamps, how="inner", on="customer_id")
    transactions_processed.dropna(inplace=True)
    return transactions_processed

trans_df = create_apk_evaluation_subset(transactions_df.copy(), Y_VALUE_PERIOD_DAYS)
trans_df

(31788324, 5) (30632391, 6) (1155933, 6)


In [ ]:
def detect_transaction_length_mismatch(input_df):
    df_copy = input_df.copy()
    df_copy["lens_article_id_x"] = df_copy["article_id_x"].apply(len)
    df_copy["lens_timestamp_x"] = df_copy["timestamp_x"].apply(len)
    df_copy["len_diff_x"] = df_copy["lens_article_id_x"] - df_copy["lens_timestamp_x"]
    diff_array_x = df_copy["len_diff_x"].to_numpy().nonzero()

    df_copy["lens_article_id_y"] = df_copy["article_id_y"].apply(len)
    df_copy["lens_timestamp_y"] = df_copy["timestamp_y"].apply(len)
    df_copy["len_diff_y"] = df_copy["lens_article_id_y"] - df_copy["lens_timestamp_y"]
    diff_array_y = df_copy["len_diff_y"].to_numpy().nonzero()

    return df_copy, diff_array_x, diff_array_y

diff_df, diff_x, diff_y = detect_transaction_length_mismatch(trans_df)
print(diff_x, diff_y)
diff_df.head()

In [ ]:
articles_df["product_type_name"].value_counts()

In [ ]:
articles_df["product_group_name"].value_counts()

In [ ]:
articles_df["colour_group_name"].value_counts()

In [ ]:
article = "0673901001"
articles_df.head()

In [ ]:
articles_df.head()

In [ ]:
articles_df_copy = articles_df.copy()
articles_df_copy.set_index("article_id", inplace=True)
articles_df_copy.head()

In [ ]:
articles_cat_dict = articles_df_copy.to_dict(orient="index")

In [ ]:
articles_cat_dict

In [ ]:
#Iterate through all y values and find their matching category.
trans_df["y_product_type_name"] = trans_df["article_id_y"].apply(lambda row : [articles_cat_dict[y_val]["product_type_name"] for y_val in row])
trans_df["y_product_group_name"] = trans_df["article_id_y"].apply(lambda row : [articles_cat_dict[y_val]["product_group_name"] for y_val in row])
trans_df["y_colour_group_name"] = trans_df["article_id_y"].apply(lambda row : [articles_cat_dict[y_val]["colour_group_name"] for y_val in row])
trans_df["y_department_name"] = trans_df["article_id_y"].apply(lambda row : [articles_cat_dict[y_val]["department_name"] for y_val in row])
trans_df.head()

In [ ]:
trans_df["article_id_x"].apply(len).value_counts().sort_index()

In [ ]:
trans_df.head()

In [ ]:
x_value_count_series = trans_df["article_id_x"].explode().value_counts()
print(x_value_count_series.shape)
x_value_count_series

In [1]:
y_value_count_series = trans_df["y_product_group_name"].explode().value_counts()
print(y_value_count_series.shape)
print(y_value_count_series.max() / y_value_count_series.sum())
y_value_count_series

NameError: name 'trans_df' is not defined

In [ ]:
trans_df["article_id_y"].explode()

In [ ]:
trans_df["article_id_x"].explode()

In [33]:
trans_df.to_csv(SAVE_LOCATION + f'_HnM_Xy_{Y_VALUE_PERIOD_DAYS}days_category_prediction.csv')